# Movie Review Analysis Outline
## import necessary python libraries


In [1]:
# "transformData.py" is this script which is intended to be run once on our data
# transform the raw data into a rdd readable format first

# import all necessary libraries
import re
import string
from operator import add
import os
import sys
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint

import pyspark.mllib.regression as mllib_reg
import pyspark.mllib.linalg as mllib_lalg
import pyspark.mllib.classification as mllib_class
import pyspark.mllib.tree as mllib_tree



print (pyspark) # test to see that pyspark is up and running okay



<module 'pyspark' from '/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/__init__.py'>


NameError: name 'spark' is not defined

# CLEAN DATA

In [2]:
# get rid of all html tags in the data
def strip_html_tags(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data.lower())

#parse and take care of funky symbols in summary and text <- normalize to root words maybe?
def cleanData (data):
    data = strip_html_tags(data)
    data =  re.sub("[\t\,\:;\(\)\"\'\~\-\!\?\`]", "",data, 0, 0)
    #data =  re.sub("[\.0`]", "",data, 0, 0) # special case to get rid of ".0" of scores -- 
    return data

#re.sub("[\.\t\,\:;\(\)\.]", " ", strip_html_tags(data.lower()), 0, 0)

# clean the data for each element of each sub list
def prepData (list_str):
    L= []
    for x in list_str:
        #print x
        L.append(cleanData(x).strip())
    return L
#####
# Notes 
# headers for the project
# [u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text']



# PARSE THE FILE WE WANT TO USE AS OUR DATA

In [3]:
# transform Data uses /// as a separator for each eleent of data for each movie review
# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky

# output < smaller_parsed

movies_txt2 = sc.textFile("50k_parsed.txt").map(lambda x: (cleanData(x).split('///')))
#movies_txt2 = sc.textFile("smaller_parsed.txt").map(lambda x: (cleanData(x).split('///')))


## Parsing out relevant fields - score and text of review

In [4]:
# prep our data

#####
# Notes 
# headers for the project
# [u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text']
# [u'productId', _ , _ , _ , u'score' [0] to get rid of .0, _  u'summary', u'text']

# take the data that we care most about
movies_new = movies_txt2.map(lambda L: (L[0], L[4][0], L[6], L[7]) if len(L) == 8 else L )
movies_new1 = movies_new.filter(lambda L: len(L) == 4) # lets just take all the data that has been parsed correctly

#print (movies_new.top(20))
print (movies_new1.count(), movies_new.count())
print (movies_new.count() == movies_new1.count())

removeHTMLTags = movies_new1.map(prepData) # remove html tags

(49994, 50000)
False


## REVISED STOP WORDS REMOVAL 

In [58]:
#REVISED STOP WORDS REMOVAL Approach

#used the stopwords file in the virtual box instead... taking too long
baseDir = os.path.join('../data')
inputPath = os.path.join('cs100', 'lab3')
STOPWORDS_PATH = 'stopwords.txt'
split_regex = r'\W+'

stopfile = os.path.join(baseDir, inputPath, STOPWORDS_PATH)
stopwords = set(sc.textFile(stopfile).collect())


def tokenize(string):
    #reusing assignment code
    """ An implementation of input string tokenization to exclude stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    splitList = re.split(split_regex, string.lower().strip())
    
    return [x for x in splitList if x!="" and x not in stopwords]  



removeSW_output= removeHTMLTags.map(lambda x: (  int(x[1]), tokenize(x[3]))  )
print removeSW_output.top(1)

[(5, [u'zulu', u'superlative', u'depiction', u'19th', u'century', u'battle', u'rourkes', u'drift', u'british', u'outpost', u'heavily', u'outnumbered', u'seige', u'zulu', u'warriors', u'fought', u'desperately', u'almost', u'last', u'man', u'somehow', u'prevailed', u'terrifying', u'exciting', u'war', u'film', u'unlike', u'splendid', u'cast', u'word', u'diamond', u'entertainment', u'tapesmy', u'experience', u'slp', u'speed', u'vhs', u'videos', u'tracks', u'remarkably', u'well', u'usually', u'recorded', u'hifi', u'sound', u'well', u'standard', u'linear', u'format', u'zulu', u'also', u'available', u'dvd', u'also', u'recommendedzulu', u'dawn', u'1979', u'tells', u'complete', u'story', u'struggles', u'tribesmen', u'british', u'soldiers', u'commenced', u'1879', u'burt', u'lancaster', u'vhs', u'edition', u'dvd', u'editionparenthetical', u'number', u'preceding', u'title', u'1', u'10', u'viewer', u'poll', u'rating', u'found', u'film', u'resource', u'website', u'8', u'0', u'zulu', u'uk1964', u'sta

## Vectorizing - making the vector of features...

In [74]:

removeSW_output= removeHTMLTags.map(lambda x: (  int(x[1]), tokenize(x[3]))  )
print removeSW_output.top(1)

[(5, [u'zulu', u'superlative', u'depiction', u'19th', u'century', u'battle', u'rourkes', u'drift', u'british', u'outpost', u'heavily', u'outnumbered', u'seige', u'zulu', u'warriors', u'fought', u'desperately', u'almost', u'last', u'man', u'somehow', u'prevailed', u'terrifying', u'exciting', u'war', u'film', u'unlike', u'splendid', u'cast', u'word', u'diamond', u'entertainment', u'tapesmy', u'experience', u'slp', u'speed', u'vhs', u'videos', u'tracks', u'remarkably', u'well', u'usually', u'recorded', u'hifi', u'sound', u'well', u'standard', u'linear', u'format', u'zulu', u'also', u'available', u'dvd', u'also', u'recommendedzulu', u'dawn', u'1979', u'tells', u'complete', u'story', u'struggles', u'tribesmen', u'british', u'soldiers', u'commenced', u'1879', u'burt', u'lancaster', u'vhs', u'edition', u'dvd', u'editionparenthetical', u'number', u'preceding', u'title', u'1', u'10', u'viewer', u'poll', u'rating', u'found', u'film', u'resource', u'website', u'8', u'0', u'zulu', u'uk1964', u'sta

In [97]:
# We will use the spark ml library for this (took a bit long to figure out)
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams

# HashingTF calulates the 'tfid feature for us
#htf = HashingTF(numFeatures=2) # features need to match dimensions of output

"""
data = removeSW_output.zipWithIndex()

keys = removeSW_output.zipWithIndex().map(lambda data: data[1] )
hashingTF = HashingTF().setInputCol("words").setOutputCol("rawFeatures").setNumFeatures(numFeatures);
tf = hashingTF.transform(keys)


#... continue from the previous example
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)


tf.cache()
idf = IDF(minDocFreq=2).fit(tf)
tfidf = idf.transform(tf)





// 4.) Create a IDFModel out of our flat vector RDD
        IDFModel idfModel = new IDF().fit(hashedData);
        // 5.) Create tfidf RDD
        JavaRDD<Vector> idf = idfModel.transform(hashedData);
        // 6.) Create Labledpoint RDD
        JavaRDD<LabeledPoint> idfTransformed = idf.zip(tupleData).map(t -> {
            return new LabeledPoint(t._2.label(), t._1);
        });
"""
############################

print "just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']"
print (htf.transform(["hi", "boo", "hi"]));

htf = HashingTF(numFeatures=200) 

hashedElements = removeSW_output.map(lambda x : htf.transform(x[1]))

labels = removeSW_output.map(lambda x : 1.0 if float(x[0]) >= 2.5 else 0.0)
#print hashedElements.top(2)
'''
[SparseVector(200, {1: 1.0, 32: 1.0, 45: 1.0, 90: 2.0, 94: 1.0, 124: 1.0, 126: 1.0, 135: 1.0, 138: 1.0, 185: 1.0, 199: 1.0}), SparseVector(200, {1: 2.0, 3: 1.0, 12: 2.0, 17: 1.0, 24: 1.0, 34: 1.0, 38: 1.0, 39: 1.0, 47: 2.0, 64: 1.0, 68: 2.0, 70: 1.0, 72: 1.0, 75: 1.0, 80: 1.0, 81: 1.0, 87: 1.0, 90: 1.0, 99: 1.0, 102: 1.0, 108: 1.0, 110: 1.0, 112: 1.0, 115: 1.0, 119: 1.0, 122: 1.0, 124: 1.0, 130: 1.0, 135: 1.0, 137: 1.0, 143: 1.0, 147: 1.0, 151: 1.0, 155: 1.0, 157: 1.0, 159: 2.0, 162: 1.0, 166: 1.0, 171: 1.0, 179: 1.0, 186: 1.0, 191: 1.0, 196: 1.0, 198: 1.0})]

'''

hashedPoints = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1]) ] if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])])))
#print hashedPoints.top(2)




hashedElements.cache()
idf = IDF().fit(hashedElements)
tfidf = idf.transform(hashedElements)


"""
IDF(minDocFreq=2).fit"""


#merge transformed data with original
merged=  labels.zip(tfidf)
print merged.top(2)
print 
feature_vector  = merged.map(lambda q: LabeledPoint(q[0], q[1])   )
print feature_vector.top(1)







#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
print feature_vector.filter(lambda x: x.label == 1.0).count()

just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']
(20,[3,7],[1.0,2.0])
[(1.0, SparseVector(200, {2: 0.7637, 9: 1.5238, 12: 1.2622, 18: 1.4201, 35: 3.0546, 39: 1.0435, 43: 0.9968, 56: 1.1372, 57: 1.5452, 60: 1.0478, 74: 3.0414, 78: 0.7582, 80: 1.103, 87: 1.144, 103: 2.5968, 113: 0.3293, 115: 1.4915, 120: 1.2875, 121: 0.8489, 124: 1.1688, 129: 1.2282, 133: 1.6184, 143: 1.7348, 144: 3.3729, 147: 0.7756, 156: 1.2945, 162: 1.3837, 169: 1.6237, 172: 5.1258, 198: 0.8439})), (1.0, SparseVector(200, {0: 1.368, 1: 1.9239, 2: 0.7637, 3: 1.0044, 4: 1.1288, 7: 1.2496, 12: 1.2622, 13: 1.5711, 14: 1.371, 15: 3.2695, 20: 4.4236, 22: 2.3038, 24: 1.5998, 27: 1.5534, 28: 1.3787, 29: 1.2386, 33: 1.1599, 35: 1.5273, 38: 1.5115, 39: 2.0869, 42: 1.3507, 43: 0.9968, 44: 1.318, 45: 1.0415, 47: 2.8539, 52: 3.6637, 53: 4.5553, 54: 1.5312, 57: 3.0905, 58: 1.5725, 59: 1.454, 60: 3.1433, 61: 1.1023, 63: 1.3662, 64: 1.2293, 66: 0.8623, 70: 1.3785, 71: 2.6234, 72: 1.3311, 74: 1.5207, 76: 1.

In [98]:
# EXPLORE DIFFERENT ML APPROACHS
# https://spark.apache.org/docs/1.1.0/api/python/pyspark.mllib.classification-module.html

##  Trying SVM approach

In [99]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel, LogisticRegressionWithLBFGS 
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.feature import HashingTF, IDF


'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - update number of features
'''


'''
dataToHash = removeSW_output.map(lambda (x,y): (x, " ".join(y)))
## Took forever to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams
htf = HashingTF(numFeatures=10) # features need to match dimensions of output

tf = htf.transform(dataToHash)

tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)


#print (tfidfIgnore.collect())
print 123

print ("SVM: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = tfidfIgnore #.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
*/
REFRESHED DATA
'''


svm_training_data, svm_testing_data= feature_vector.randomSplit([0.7, 0.3])


#LogisticRegressionWithLBFGS 
# Build the Support Vector Machine model
svm_model = SVMWithSGD.train(svm_training_data, iterations=500) # lower iterations so we arent here forever

# Evaluate the model on training data
svm_labelsAndPreds = svm_training_data.map(lambda p: (p.label, svm_model.predict(p.features)))
svm_trainErr = 100* svm_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(svm_training_data.count())
print("training error of the SVM model: " + "%.3f" %(svm_trainErr))
print("training accuracy of the SVM model: " + "%.3f" %(100-svm_trainErr))


#Evaluate on testing data.. the correct way lolz

svm_labelsAndPredsTest = svm_testing_data.map(lambda p: (p.label, svm_model.predict(p.features)))
svm_testErr = 100* svm_labelsAndPredsTest.filter(lambda (v, p): v != p).count() / float(svm_testing_data.count())
print("testing error of the SVM model: " + "%.3f" %(svm_testErr))
print("training accuracy of the SVM model: " + "%.3f" %(100-svm_testErr))






training error of the SVM model: 14.490
training accuracy of the SVM model: 85.510
testing error of the SVM model: 14.147
training accuracy of the SVM model: 85.853


In [100]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS , LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

log_training_data, log_testing_data= feature_vector.randomSplit([0.6, 0.4])

#LogisticRegressionWithLBFGS 
# Build the Support Vector Machine model
log_model = LogisticRegressionWithLBFGS.train(log_training_data, iterations=500) # lower iterations so we arent here forever

# Evaluate the model on training data
log_labelsAndPreds = log_training_data.map(lambda p: (p.label, log_model.predict(p.features)))
log_trainErr = 100* log_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(log_training_data.count())
print "Log Training Accuracy: " + "%.3f" %(100-log_trainErr) + "%" 
print("training error of the LogisticRegressionWithLBFGS model= " + "%.3f" % (log_trainErr)) + "%" 

#Evaluate on testing data.. the correct way lolz

log_labelsAndPredsTest = log_testing_data.map(lambda p: (p.label, log_model.predict(p.features)))
log_testErr = 100* log_labelsAndPredsTest.filter(lambda (v, p): v != p).count() / float(log_testing_data.count())

print "Log Testing Accuracy: " + "%.3f" %(100-log_testErr) + "%" 
print("testing error of the LogisticRegressionWithLBFGS model: " + "%.3f" %(log_testErr)) + "%" 


Log Training Accuracy: 85.156%
training error of the LogisticRegressionWithLBFGS model= 14.844%
Log Testing Accuracy: 84.922%
testing error of the LogisticRegressionWithLBFGS model: 15.078%


## Do some Predictions - Bayes

In [101]:
### THIS USES NAIVE BAYES CLASSIFIER
### with 116 elements with the first output.txt 86KB , gets 80-93%
# tested with lots of data.... - 111MB



# would be nice to figure out which division is the best
#training_data, testing_data= feature_vector.randomSplit([0.6, 0.4])
bayes_training_data, bayes_testing_data= feature_vector.randomSplit([0.7, 0.3])

# parameters:
lamda = 1.0

# build Naive Bayes Classifier
nbay = mllib_class.NaiveBayes.train(bayes_training_data, lamda)

# Make prediction and test accuracy.




bayes_labelsAndPreds = bayes_training_data.map(lambda p : (nbay.predict(p.features), p.label))
#bayes_trainErr = bayes_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(bayes_training_data.count())
bayes_trainErr = 100.0 * bayes_labelsAndPreds.filter(lambda (x, v): x == v).count() / bayes_training_data.count()

print "Naive Bayes Training Accuracy: " + "%.3f" %(bayes_trainErr) + "%" 
print "Bayes Training error: " + "%.3f" % (100-bayes_trainErr) + "%" 


# Testing Data Accuracy

bayes_labelsAndPreds = bayes_testing_data.map(lambda p : (nbay.predict(p.features), p.label))
#bayes_testErr = bayes_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(bayes_testing_data.count())
bayes_testErr = 100.0 * bayes_labelsAndPreds.filter(lambda (x, v): x == v).count() / bayes_testing_data.count()


print "Naive Bayes Testing Accuracy: " + "%.3f" %(bayes_testErr) + "%" 
print "Bayes Testing error: " + "%.3f" % (100-bayes_testErr) + "%" 

Naive Bayes Training Accuracy: 85.124%
Bayes Training error: 14.876%
Naive Bayes Testing Accuracy: 85.464%
Bayes Testing error: 14.536%


In [105]:
# PREDICTION - LETS USE OUR MODEL 


# first apply tokenization

#newdata = sc.textFile("smaller_predictData.txt").map(lambda x: (cleanData(x).split('///')))
#newdata = sc.textFile("100k_parsed.txt").map(lambda x: (cleanData(x).split('///')))

# parse data for the params we want

strippedData = newdata.map(lambda L: (L[0], float(L[4]), L[6], L[7]) if len(L) == 8 else L )
completeData = strippedData.filter(lambda L: len(L) == 4)
print (strippedData.count())
print (completeData.count())
print (strippedData.count() == completeData.count())


movieData = completeData.map(lambda x: (  x[1], tokenize(x[3]))   )

#.filter(lambda x: float (x) >= 2.5).collect())
#print movieData.filter(lambda x: float(x[1]) >= 2.5).top(10)
#print movieData.filter(lambda x: float(x[1]) >= 2.5).count()


100000
99991
False


In [106]:
1.0 == 1.0

True

In [117]:
# HASH OUR DATA TO PREDICT

#htf = HashingTF(numFeatures=200) 
predict_hashedElements = movieData.map(lambda x : htf.transform(x[1]))

predict_labels = movieData.map(lambda x : 1.0 if float(x[0]) >= 2.5 else 0.0)
#print hashedElements.top(2)
'''
[SparseVector(200, {1: 1.0, 32: 1.0, 45: 1.0, 90: 2.0, 94: 1.0, 124: 1.0, 126: 1.0, 135: 1.0, 138: 1.0, 185: 1.0, 199: 1.0}), SparseVector(200, {1: 2.0, 3: 1.0, 12: 2.0, 17: 1.0, 24: 1.0, 34: 1.0, 38: 1.0, 39: 1.0, 47: 2.0, 64: 1.0, 68: 2.0, 70: 1.0, 72: 1.0, 75: 1.0, 80: 1.0, 81: 1.0, 87: 1.0, 90: 1.0, 99: 1.0, 102: 1.0, 108: 1.0, 110: 1.0, 112: 1.0, 115: 1.0, 119: 1.0, 122: 1.0, 124: 1.0, 130: 1.0, 135: 1.0, 137: 1.0, 143: 1.0, 147: 1.0, 151: 1.0, 155: 1.0, 157: 1.0, 159: 2.0, 162: 1.0, 166: 1.0, 171: 1.0, 179: 1.0, 186: 1.0, 191: 1.0, 196: 1.0, 198: 1.0})]

'''



predict_hashedElements.cache()
pidf = IDF().fit(predict_hashedElements)
ptfidf = pidf.transform(predict_hashedElements)


"""
IDF(minDocFreq=2).fit"""


#merge transformed data with original
predict_merged=  predict_labels.zip(ptfidf)
print predict_merged.top(2)
print 
featurePV  = predict_merged.map(lambda q: LabeledPoint(q[0], q[1])   )
print featurePV.top(1)







#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
print featurePV.filter(lambda x: x.label == 1.0).count()

# 0.8588372954

[(1.0, SparseVector(200, {0: 1.374, 2: 0.7789, 5: 1.432, 9: 1.5184, 12: 0.6387, 20: 1.4818, 21: 1.2526, 22: 1.1598, 25: 1.4449, 29: 1.2381, 39: 1.0625, 44: 1.3003, 47: 0.9655, 50: 1.1621, 66: 0.8489, 68: 0.7608, 76: 1.3512, 78: 2.2766, 80: 1.0936, 88: 1.3486, 90: 2.4002, 109: 1.5145, 111: 1.5593, 113: 1.3248, 118: 1.5782, 121: 0.8357, 135: 0.723, 141: 1.1385, 151: 1.2973, 170: 1.1783, 185: 1.0151, 186: 1.9724, 188: 1.0745})), (1.0, SparseVector(200, {14: 1.3862, 27: 1.5589, 28: 1.3668, 36: 1.746, 43: 0.9986, 64: 4.9144, 68: 0.7608, 88: 2.6973, 90: 0.8001, 113: 0.3312, 121: 0.8357, 124: 1.1648, 125: 1.4608, 127: 1.1862, 129: 1.2287, 135: 0.723, 146: 3.2018, 147: 0.781, 172: 1.7086, 185: 1.0151, 190: 1.757}))]

[LabeledPoint(1.0, (200,[12,18,44,47,50,66,69,73,85,88,100,104,147,160,170,186,189],[0.638654752521,1.40327913655,1.30025686941,0.965453495252,2.32428647386,0.848902608918,1.30899409113,1.38321884894,1.67144610155,1.34864886003,1.43974172494,1.08771104684,1.56200523506,1.180078623

In [108]:
'''   
# HashingTF calulates the tfid feature for us
htf = HashingTF(numFeatures=1) # features need to match dimensions of output

featurePV = movieData.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))

print featurePV.filter(lambda x: x.label == 1.0).count()
print featurePV.count()


print featurePV.top(2)
#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
#print "Number of Positives: ", feature_vector.filter(lambda x: x.label == 1.0).count()
#print "Total Number of Data Points ", feature_vector.filter(lambda x: x.label == 1.0).count()

'''

'''
sameModel = SVMModel.load(sc, output_dir)
predictionAndLabel = test.map(lambda p: (sameModel.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / test.count()
'''

'   \n# HashingTF calulates the tfid feature for us\nhtf = HashingTF(numFeatures=1) # features need to match dimensions of output\n\nfeaturePV = movieData.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))\n\nprint featurePV.filter(lambda x: x.label == 1.0).count()\nprint featurePV.count()\n\n\nprint featurePV.top(2)\n#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))\n#print "Number of Positives: ", feature_vector.filter(lambda x: x.label == 1.0).count()\n#print "Total Number of Data Points ", feature_vector.filter(lambda x: x.label == 1.0).count()\n'

In [113]:
# SVM PREDICTIONS

print (svm_model) 
"""
(weights=[0.0368997186104,0.0592059564322,0.0511764523985,0.0944362393672,-0.1234655326,-0.0213073956774,0.0598834383627,0.0301296198158,0.0071253205336,-0.0773313907846,0.135198560493,0.0150622838296,0.252044696973,-0.0485074421553,0.0472279727127,-0.0602539313766,0.0113035109407,0.0384187101225,-0.188469809917,0.00267308133624,-0.0642051128667,0.0501615460127,0.185930140409,-0.0366052969578,0.056863287584,0.0493490204659,0.0300440006011,-0.0113687899148,-0.0502989129382,-0.00495112968516,0.0705056106545,0.0723744391356,-0.00780960280399,0.0648763892711,-0.0142119736707,-0.0101463323161,-0.0950565271662,-0.025407627537,0.0165633108803,0.0726917340991,-0.0543413768729,-0.11493914306,0.048542911942,-0.00490814080509,0.00183017415496,-0.0154270882776,-0.0205600111133,0.190822611363,-0.0085785906532,0.00292457229289,0.0908497832715,-0.0306247814764,-0.0272281652519,-0.00461787021553,-0.0155235065268,0.0296160453275,-0.139970211245,0.0709340007922,0.0141226375075,0.0767565407429,0.0923194359137,-0.0793072158897,-0.162151121072,-0.00582485572825,-0.027916288075,-0.0210472483295,0.00345838293048,0.013724710527,0.101103869025,-0.0297336253554,0.0279691564116,0.0689764908622,-0.13494363691,-0.0668335425458,-0.060322739572,-0.0956377298681,0.0769876814833,0.0117181091065,0.0126856313014,0.109426784166,0.0861160290819,-0.0244508540554,0.0161106487066,0.0400982188816,-0.00548897974707,-0.019190100889,0.00461539237948,0.149560544582,0.0121178683009,-0.0639882247701,0.529805130452,-0.0128682615876,0.00480636850342,-0.00410932632816,0.0351692687322,-0.0198583418872,-0.0229548568145,-0.00340860804861,0.0333486117234,-0.0330106365069,0.0412798754816,0.0537416796427,0.0374752509553,0.0180206899498,-0.00407131571665,-0.0479947909314,-0.0435645227459,-0.00546051146944,-0.0673355134481,-0.0122398316702,-0.0386454914434,0.0319355800185,-0.0641061556095,0.285542644392,-0.0755651792838,-0.0206629484963,0.023072234171,0.0700045307581,-0.0248450979802,0.0345836657542,0.043560324962,0.189977075545,-0.0267604623301,-0.134677417127,0.0549936914647,0.0470644285613,0.0516906946357,0.0294428387543,0.0647815568887,0.0564883632182,-0.105579244454,0.00852526947916,0.00525727645833,-0.0596530787731,-0.0415343964404,0.11638812774,0.0968259524311,0.208324958134,-0.00117460116655,-0.0505781712885,-0.0323338532613,-0.132213597077,-0.0048034943035,0.125680763706,0.00720316225617,0.0482061313933,-0.0819678955725,0.225878458733,-0.0487156611568,0.0450780570356,0.113355924008,-0.0311896686759,-0.0509124818306,0.036520188844,-0.0269550841676,-0.0107445622353,0.0824359871593,-0.0148080705468,0.0813362368022,0.00962483814534,0.0986313594561,0.0438001312644,0.0281565279201,0.023460809598,-0.0669994037871,-0.0285583255943,-0.0715752292781,-0.0355364452118,0.0191930116858,0.0161009483418,-0.0956563973472,0.141158315143,0.0294657105506,-0.0151626852378,0.0865943223035,-0.0415571028551,-0.0342841059177,-0.0237316978741,-0.0302115070006,-0.00182684799626,0.00168597665351,-0.0041243305443,0.0217597781446,-0.0846252980175,-0.013768106553,0.12940121048,0.0437548868609,0.0418497207055,0.123918612503,-0.0535942910674,0.0279919178596,0.0752968777908,-0.0366714491301,-0.00508041992326,0.0590876803126,-0.116625416506,-0.0194474547958,-0.0545133996421,0.023332139766,-0.014695114347], intercept=0.0)
"""
#print (log_model) 
#print (nbay)




svm_predictions = featurePV.map(lambda p: (p.label, svm_model.predict(p.features)))
svm_PErr = 100* svm_predictions.filter(lambda (v, p): v != p).count() / float(featurePV.count())
print "SVM Prediction Accuracy: " + "%.3f" %(svm_PErr) + "%" 
print "SVM  Prediction error: " + "%.3f" % (100-svm_PErr) + "%" 



(weights=[0.0368997186104,0.0592059564322,0.0511764523985,0.0944362393672,-0.1234655326,-0.0213073956774,0.0598834383627,0.0301296198158,0.0071253205336,-0.0773313907846,0.135198560493,0.0150622838296,0.252044696973,-0.0485074421553,0.0472279727127,-0.0602539313766,0.0113035109407,0.0384187101225,-0.188469809917,0.00267308133624,-0.0642051128667,0.0501615460127,0.185930140409,-0.0366052969578,0.056863287584,0.0493490204659,0.0300440006011,-0.0113687899148,-0.0502989129382,-0.00495112968516,0.0705056106545,0.0723744391356,-0.00780960280399,0.0648763892711,-0.0142119736707,-0.0101463323161,-0.0950565271662,-0.025407627537,0.0165633108803,0.0726917340991,-0.0543413768729,-0.11493914306,0.048542911942,-0.00490814080509,0.00183017415496,-0.0154270882776,-0.0205600111133,0.190822611363,-0.0085785906532,0.00292457229289,0.0908497832715,-0.0306247814764,-0.0272281652519,-0.00461787021553,-0.0155235065268,0.0296160453275,-0.139970211245,0.0709340007922,0.0141226375075,0.0767565407429,0.09231943

In [118]:
# Log

log_predictions = featurePV.map(lambda p: (p.label, log_model.predict(p.features)))
log_PErr = 100* log_predictions.filter(lambda (v, p): v != p).count() / float(featurePV.count())
print "Log Regression Prediction Accuracy: " + "%.3f" %(log_PErr) + "%" 
print "Log Regression  Prediction error: " + "%.3f" % (100-log_PErr) + "%"


Log Regression Prediction Accuracy: 15.500%
Log Regression  Prediction error: 84.500%


In [119]:
# Bayes
bayes_predictions = featurePV.map(lambda p : (nbay.predict(p.features), p.label))
bayes_PErr = 100.0 * bayes_predictions.filter(lambda (x, v): x == v).count() / featurePV.count()
print "Naive Bayes Prediction Accuracy: " + "%.3f" %(bayes_PErr) + "%" 
print "Bayes Prediction error: " + "%.3f" % (100-bayes_PErr) + "%" 


Naive Bayes Prediction Accuracy: 84.451%
Bayes Prediction error: 15.549%
